In [73]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
import warnings
warnings.simplefilter("ignore")
import statsmodels.api as sm
import nltk
import ast


In [71]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.preprocessing import LabelEncoder
from sklearn import set_config
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score,precision_score,recall_score,roc_auc_score

#-----------MODELS----------------------------------------
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgboost
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from tpot import TPOTClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,BaggingClassifier


C:\Users\Shailesh\anaconda4\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [3]:
df= pd.read_csv('breast-cancer-wisconsin.csv')

In [4]:
df.head()

,CodeNumber,ClumpThickness,UniformityCellSize,UniformityCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,CancerType
0,1002945,5,4,4,5,7,10,3,2,1,2
1,1015425,3,1,1,1,2,2,3,1,1,2
2,1016277,6,8,8,1,3,4,3,7,1,2
3,1017023,4,1,1,3,2,1,3,1,1,2
4,1017122,8,10,10,8,7,10,9,7,1,4


In [5]:
df.columns

Index(['CodeNumber', 'ClumpThickness', 'UniformityCellSize',
       'UniformityCellShape', 'MarginalAdhesion', 'SingleEpithelialCellSize',
       'BareNuclei', 'BlandChromatin', 'NormalNucleoli', 'Mitoses',
       'CancerType'],
      dtype='object')

In [7]:
df.value_counts().sum()

698

In [9]:
df.isnull().sum()

CodeNumber                  0
ClumpThickness              0
UniformityCellSize          0
UniformityCellShape         0
MarginalAdhesion            0
SingleEpithelialCellSize    0
BareNuclei                  0
BlandChromatin              0
NormalNucleoli              0
Mitoses                     0
CancerType                  0
dtype: int64

In [20]:
df.value_counts().sum()

698

In [22]:
df.dtypes

CodeNumber                   int64
ClumpThickness               int64
UniformityCellSize           int64
UniformityCellShape          int64
MarginalAdhesion             int64
SingleEpithelialCellSize     int64
BareNuclei                  object
BlandChromatin               int64
NormalNucleoli               int64
Mitoses                      int64
CancerType                   int64
dtype: object

In [48]:
df[df['BareNuclei']=='?']

,CodeNumber,ClumpThickness,UniformityCellSize,UniformityCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses,CancerType
22,1057013,8,4,5,1,2,?,7,3,1,4
39,1096800,6,6,6,9,6,?,7,8,1,2
138,1183246,1,1,1,1,1,?,2,1,1,2
144,1184840,1,1,3,1,2,?,2,1,1,2
157,1193683,1,1,2,1,3,?,1,1,1,2
163,1197510,5,1,1,1,2,?,3,1,1,2
234,1241232,3,1,4,1,2,?,3,1,1,2
248,169356,3,1,1,1,2,?,3,1,1,2
274,432809,3,1,3,1,2,?,2,1,1,2
291,563649,8,8,8,1,2,?,6,10,1,4


In [51]:
df['BareNuclei'].value_counts()

1     401
10    132
2      30
5      30
3      28
8      21
4      19
?      16
9       9
7       8
6       4
Name: BareNuclei, dtype: int64

In [55]:
df['BareNuclei']=np.where(df['BareNuclei']=='?',df['BareNuclei'].mode(),df['BareNuclei'])

In [57]:
df['BareNuclei'].value_counts()

1     417
10    132
2      30
5      30
3      28
8      21
4      19
9       9
7       8
6       4
Name: BareNuclei, dtype: int64

In [10]:
df.describe()

,CodeNumber,ClumpThickness,UniformityCellSize,UniformityCellShape,MarginalAdhesion,SingleEpithelialCellSize,BlandChromatin,NormalNucleoli,Mitoses,CancerType
count,6.980000e+02,698.000000,698.000000,698.000000,698.000000,698.000000,698.000000,698.000000,698.000000,698.000000
mean,1.071807e+06,4.416905,3.137536,3.210602,2.809456,3.217765,3.438395,2.869628,1.590258,2.690544
std,6.175323e+05,2.817673,3.052575,2.972867,2.856606,2.215408,2.440056,3.055004,1.716162,0.951596
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.702582e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238354e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [59]:
X=df.drop(columns=['CancerType'])
y=df['CancerType']

In [60]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)

In [61]:
y_test.shape

(175,)

In [62]:
model=RandomForestClassifier()


In [63]:
model.fit(X_train,y_train)

RandomForestClassifier()

In [64]:
y_pred=model.predict(X_test)

In [65]:
accuracy_score(y_test,y_pred)

0.9485714285714286